In [1]:
import pandas as pd
import numpy as np
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix)
from keras import Sequential
from keras.layers import Dense, LSTM, Activation, Dropout
from keras_tuner.tuners import GridSearch

2024-10-31 22:46:57.825601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 22:46:57.841252: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 22:46:57.846131: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 22:46:57.860536: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 22:46:58.738853: W tensorflow/compiler/tf2

In [2]:
seed = 42
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)
tf.keras.utils.set_random_seed(seed)

In [3]:
# Load Data
df = pd.read_csv('dataset_phishing.csv')

In [4]:
#Top 27 features from feature selection
features = ['shortest_word_path',
 'ratio_intMedia',
 'links_in_tags',
 'nb_hyphens',
 'page_rank',
 'avg_word_path',
 'ratio_extHyperlinks',
 'longest_words_raw',
 'google_index',
 'length_hostname',
 'longest_word_host',
 'domain_registration_length',
 'nb_www',
 'nb_underscore',
 'nb_dots',
 'ratio_extMedia',
 'phish_hints',
 'domain_in_title',
 'web_traffic',
 'safe_anchor',
 'nb_space',
 'shortening_service',
 'ip',
 'domain_age',
 'nb_qm',
 'nb_hyperlinks',
 'nb_slash']

In [5]:
X = df[features]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Reshape features with 1 timestep to fit into RNN
X_reshaped = X_scaled.reshape((X.shape[0],1,X.shape[1]))  

le = LabelEncoder()
y = le.fit_transform(df["status"])
X_train, X_temp, y_train, y_temp = train_test_split(X_reshaped, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [6]:
#Define model to fir into GridSearch later
def build_model(hp):
    model = Sequential()
    model.add(LSTM(hp.Choice('units',[128,512,1024,2048]), activation=hp.Choice('activation',['relu','tanh']),input_shape=(1,X.shape[1]))) #4 Different layer sizes
    model.add(Dropout(hp.Float('dropout',0.1,0.5,step=0.1))) #5 different dropout values
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [7]:
#Grid search for best hyperparameters
# tuner = GridSearch(
#     build_model,
#     objective='val_accuracy',
#     max_trials=20
# )
# tuner.search(X_train,y_train,epochs=10,validation_data=(X_val,y_val))
best_model = Sequential()
best_model.add(LSTM(1024, activation='relu',input_shape=(1,X.shape[1]))) 
best_model.add(Dropout(0.1))
best_model.add(Dense(1, activation='sigmoid'))
best_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# best_model = tuner.get_best_models()[0]
# best_hp = tuner.get_best_hyperparameters()[0]
# print(best_hp.values)
# tuner.results_summary()

history = best_model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_val,y_val))

y_pred = (best_model.predict(X_test) > 0.5)

I0000 00:00:1730386020.423726 1079323 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730386020.460307 1079323 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730386020.462716 1079323 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1730386020.464823 1079323 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/5


I0000 00:00:1730386023.622458 1079367 service.cc:146] XLA service 0x7fb5ac0d37a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730386023.622496 1079367 service.cc:154]   StreamExecutor device (0): NVIDIA A40, Compute Capability 8.6
2024-10-31 22:47:03.655965: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-31 22:47:03.862612: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-31 22:47:06.054980: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_50', 36 bytes spill stores, 36 bytes spill loads

2024-10-31 22:47:07.080516: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion

216/251 ━━━━━━━━━━━━━━━━━━━━ 0s 771us/step - accuracy: 0.9085 - loss: 0.3338

I0000 00:00:1730386028.853630 1079367 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


251/251 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.9117 - loss: 0.3184 - val_accuracy: 0.9452 - val_loss: 0.1547
Epoch 2/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 967us/step - accuracy: 0.9442 - loss: 0.1456 - val_accuracy: 0.9463 - val_loss: 0.1434
Epoch 3/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 980us/step - accuracy: 0.9502 - loss: 0.1281 - val_accuracy: 0.9504 - val_loss: 0.1370
Epoch 4/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 953us/step - accuracy: 0.9569 - loss: 0.1167 - val_accuracy: 0.9516 - val_loss: 0.1331
Epoch 5/5
251/251 ━━━━━━━━━━━━━━━━━━━━ 0s 987us/step - accuracy: 0.9594 - loss: 0.1098 - val_accuracy: 0.9539 - val_loss: 0.1308
54/54 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step 


In [8]:
print(f"Accuracy: {accuracy_score(y_test,y_pred):.4f}")
print(f"Precision: {precision_score(y_test,y_pred):.4f}")
print(f"Recall:  {recall_score(y_test,y_pred):.4f}")
print(f"F1 Score: {f1_score(y_test,y_pred):.4f}")
print(f"ROC_AUC Score: {roc_auc_score(y_test,y_pred):.4f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.9644
Precision: 0.9671
Recall:  0.9615
F1 Score: 0.9643
ROC_AUC Score: 0.9644
              precision    recall  f1-score   support

           0       0.96      0.97      0.96       858
           1       0.97      0.96      0.96       857

    accuracy                           0.96      1715
   macro avg       0.96      0.96      0.96      1715
weighted avg       0.96      0.96      0.96      1715

